In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# functions

In [2]:
def caf(tt,k1,k2,k3,k4,k5,k6,k7,k8,par0,bro0,phy0):
    '''Takes timepoints and kinetic parameters, calculates concentration time series of caffeine.'''
    k9 = k2+k3+k4+k5
    y = k1/(k9-k1)*(np.exp(-k1*tt)-np.exp(-k9*tt))
    return y

def par(tt,k1,k2,k3,k4,k5,k6,k7,k8,par0,bro0,phy0):
    '''Takes timepoints and kinetic parameters, calculates concentration time series of paraxanthine.'''
    k9 = k2+k3+k4+k5
    y = k1*k2/(k9-k1)*(np.exp(-k9*tt)/(k9-k6)-np.exp(-k1*tt)/(k1-k6)+np.exp(-k6*tt)*(k9-k1)/((k9-k6)*(k1-k6)))+par0*np.exp(-k6*tt)
    return y

def bro(tt,k1,k2,k3,k4,k5,k6,k7,k8,par0,bro0,phy0):
    '''Takes timepoints and kinetic parameters, calculates concentration time series of theobromine.'''
    k9 = k2+k3+k4+k5
    y = k1*k3/(k9-k1)*(np.exp(-k9*tt)/(k9-k7)-np.exp(-k1*tt)/(k1-k7)+np.exp(-k7*tt)*(k9-k1)/((k9-k7)*(k1-k7)))+bro0*np.exp(-k7*tt)
    return y

def phy(tt,k1,k2,k3,k4,k5,k6,k7,k8,par0,bro0,phy0):
    '''Takes timepoints and kinetic parameters, calculates concentration time series of theophylline.'''
    k9 = k2+k3+k4+k5
    y = k1*k4/(k9-k1)*(np.exp(-k9*tt)/(k9-k8)-np.exp(-k1*tt)/(k1-k8)+np.exp(-k8*tt)*(k9-k1)/((k9-k8)*(k1-k8)))+phy0*np.exp(-k8*tt)
    return y


# extract MRE and CV for time-independent comparisons

In [3]:
def extract_quality_parameters(run_list):
    rounder = 2          # decimal places to round numbers
    rep_lim  = 3000      # maximum number of bootstrap replicates taken into account

    overview = {}
    info = pd.read_csv('run_information.csv',index_col=0)
    info = info.transpose()

    for run in run_list:
        # write general fit information
        overview[run+' CV'] = {}
        overview[run+' MRE'] = {}
        for information in ['mc_replicates','c0','loss','error%']:
            overview[run+' MRE'][information.strip(' ')] = info.loc[information,run]
            overview[run+' CV'][information.strip(' ')] = info.loc[information,run]

        # load fitted data
        with open(f'runs_manuscript/{run}.txt','r') as file:
            lines = file.readlines()
        parameters = []
        fits       = []
        timepoints = []
        sv_values  = []
        for line in lines:
            if 'ORIGINAL' in line:
                for i in line.split():
                    if i[0].isnumeric():
                        parameters.append(float(i))
            elif 'TIMEPOINTS' in line:
                for i in line.split():
                    if i[0].isnumeric():
                        timepoints.append(float(i))
            elif 'SV_VALUES' in line:
                tmp = []
                for i in line.split():
                    if i[0].isnumeric():
                        tmp.append(float(i))
                sv_values.append(tmp)
            elif line[0].isnumeric():
                tmp = []
                for i in line.split():
                    tmp.append(float(i))
                fits.append(tmp)

        fits      = fits[:rep_lim]
        sv_values = sv_values[:rep_lim]
        # write more general data
        overview[run+' CV']['n_samples'] = len(fits)
        overview[run+' CV']['n_timepoints'] = len(timepoints)
        overview[run+' MRE']['n_samples'] = len(fits)
        overview[run+' MRE']['n_timepoints'] = len(timepoints)


        # extract kinetic data
        timepoints = np.array(timepoints)
        fits       = np.array(fits)                
        sv_values  = np.array(sv_values)
        parameter_names = ['kcincaf','kcafpar','kcafbro','kcafphy','kcafdeg','kpardeg','kbrodeg','kphydeg','par0','bro0','phy0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24']
        label_names     = []
        for i in range(len(parameters)):
            sigma  = np.sqrt(np.sum((fits[:,i]-parameters[i])**2)/fits.shape[0])
            median = np.median((fits[:,i]-parameters[i])/parameters[i])
            s      = np.std(fits[:,i])
            rel_sigma = sigma/parameters[i]*100
            rel_s     = s/parameters[i]*100
            overview[run+' CV'][parameter_names[i]] = round(rel_sigma,rounder)
            overview[run+' MRE'][parameter_names[i]] = round(median*100,rounder)

        # extract sweat volume data
        sv_rel_diffs = []
        sv_abs_diffs = []
        outlier_number = 0
        for i in range(fits.shape[0]):
            tmp_diffs     = fits[i,11:]-sv_values[i]
            tmp_rel_diffs = tmp_diffs/sv_values[i]
            for i in tmp_rel_diffs:
                if i < 10:
                    sv_rel_diffs.append(i)
                else:
                    outlier_number += 1
            sv_abs_diffs.append(tmp_diffs)
        print(f'{run}: Removed {outlier_number} outlier(s) of SV statistics. This is {round(outlier_number/len(np.array(sv_values).flatten())*100,5)} % of the whole dataset.')
        sv_rel_diffs = np.array(sv_rel_diffs).flatten()
        sv_abs_diffs = np.array(sv_abs_diffs).flatten()
        rel_sigma = np.sqrt(np.sum(sv_rel_diffs**2)/len(sv_rel_diffs))
        rel_s     = np.std(sv_rel_diffs)
        abs_sigma = np.sqrt(np.sum(sv_abs_diffs**2)/len(sv_abs_diffs))
        median_rel = np.median(sv_rel_diffs)
        overview[run+' CV']['sv'] = round(rel_sigma*100,rounder)
        overview[run+' MRE']['sv'] = round(median_rel*100,rounder)

        # extract concentration data
        for func, name in zip([caf,par,bro,phy],['caf','par','bro','phy']):
            true_conc     = func(timepoints,*parameters)
            fitted_matrix = []
            for i in range(fits.shape[0]):
                fitted = func(timepoints,*fits[i,:11])
                fitted_matrix.append(fitted)
            fitted_matrix = np.array(fitted_matrix)

            rel_sigma_list = []
            rel_s_list     = []
            median_list    = []
            for i in range(fitted_matrix.shape[1]):
                abs_sigma = np.sqrt(np.sum((fitted_matrix[:,i]-true_conc[i])**2)/fitted_matrix.shape[0])
                abs_s     = np.std(fitted_matrix[:,i])
                median_list.append((fitted_matrix[:,i]-true_conc[i])/true_conc[i])
                rel_sigma = abs_sigma/true_conc[i]*100
                rel_s     = abs_s/true_conc[i]*100
                rel_sigma_list.append(rel_sigma)
                rel_s_list.append(rel_s)
            overview[run+' CV'][name+' conc.'] = round(np.nanmean(rel_sigma_list[1:]),rounder)
            median = np.nanmedian(median_list)
            overview[run+' MRE'][name+' conc.'] = round(median*100,rounder)

    # add an average row
    for run in overview:
        tmp = []
        for i in overview[run]:
            if i not in ['mc_replicates','c0','loss','n_samples','n_timepoints','error%']:
                tmp.append(overview[run][i])
        overview[run]['total_avg'] = round(np.mean(np.abs(tmp)),rounder)

    # put everything in a nice DataFrame
    overview_df = pd.DataFrame(overview)
    # save as csv
    # overview_df.to_csv('analysis_tables/v_sweat_norm.csv')

    # make column lists for MRE and CV for easier view
    CV  = []
    MRE = []
    for i in overview_df.columns:
        if 'MRE' in i:
            MRE.append(i)
        else:
            CV.append(i)
    return MRE, CV, overview_df

In [4]:
### Supplementary Notes: Sensitivity Analysis
#   Following Comparisons have been done
### 

    # different loss functions
losses    = ['run_104','run_108','run_109','run_110','run_111','run_112','run_113','run_121']
    # different MC replicates
motecarlo = ['run_138','run_123','run_121','run_124']
    # how many timepoints to use for fitting?
timepoints = ['run_121','run_125','run_126','run_127','run_128','run_129','run_130']
    # different error estimates
errorsize = ['run_133','run_121','run_131','run_135']
    # difference with without SV normalization
v_sweat_norm   = ['run_136','run_134']


MRE,CV, overview = extract_quality_parameters(losses) # you can try all different lists

run_104: Removed 1 outlier(s) of SV statistics. This is 0.01667 % of the whole dataset.
run_108: Removed 2 outlier(s) of SV statistics. This is 0.03333 % of the whole dataset.


/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in true_divide
/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:108: RuntimeWarning: invalid value encountered in double_scalars
/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: invalid value encountered in double_scalars


run_109: Removed 0 outlier(s) of SV statistics. This is 0.0 % of the whole dataset.
run_110: Removed 3 outlier(s) of SV statistics. This is 0.05 % of the whole dataset.
run_111: Removed 3 outlier(s) of SV statistics. This is 0.05 % of the whole dataset.
run_112: Removed 5 outlier(s) of SV statistics. This is 0.08333 % of the whole dataset.
run_113: Removed 1 outlier(s) of SV statistics. This is 0.01667 % of the whole dataset.
run_121: Removed 2 outlier(s) of SV statistics. This is 0.03333 % of the whole dataset.


In [5]:
overview
overview[MRE]
overview[CV]

,run_104 CV,run_108 CV,run_109 CV,run_110 CV,run_111 CV,run_112 CV,run_113 CV,run_121 CV
mc_replicates,100,100,100,100,100,100,100,100
c0,yes,yes,yes,yes,yes,yes,yes,yes
loss,cauchy,huber,soft_l1,robust,max_cauchy,max_huber,max_soft_l1,max_robust
error%,10,10,10,10,10,10,10,10
n_samples,300,300,300,300,300,300,300,300
n_timepoints,20,20,20,20,20,20,20,20
kcincaf,39.19,35.8,35.15,36.75,29.79,30.01,32.49,25.56
kcafpar,25.48,23.48,23.85,16.7,18.2,17.76,16.81,19.08
kcafbro,24.52,23.1,22.71,23.93,15.94,16.12,15.61,19.08
kcafphy,24.81,22.62,23.13,24.18,16.89,15.76,15.65,18.44


# extract time-dependent MRE and CV

In [6]:
def extract_quality_parameters_time(run_list):
    overview = {}
    info = pd.read_csv('run_information.csv',index_col=0)
    info = info.transpose()
    rounder = 2
    rep_lim  = 3000

    for run in run_list:
        overview[run] = {}
        overview[run+' median'] = {}


        for information in ['mc_replicates','c0','loss','error%']:
            overview[run+' median'][information.strip(' ')] = info.loc[information,run]
            overview[run][information.strip(' ')] = info.loc[information,run]

        # LOAD DATA
        with open(f'runs_manuscript/{run}.txt','r') as file:
            lines = file.readlines()
        parameters = []
        fits       = []
        timepoints = []
        sv_values  = []
        for line in lines:
            if 'ORIGINAL' in line:
                for i in line.split():
                    if i[0].isnumeric():
                        parameters.append(float(i))
            elif 'TIMEPOINTS' in line:
                for i in line.split():
                    if i[0].isnumeric():
                        timepoints.append(float(i))
            elif 'SV_VALUES' in line:
                tmp = []
                for i in line.split():
                    if i[0].isnumeric():
                        tmp.append(float(i))
                sv_values.append(tmp)
            elif line[0].isnumeric():
                tmp = []
                for i in line.split():
                    tmp.append(float(i))
                fits.append(tmp)

        fits      = fits[:rep_lim]
        sv_values = sv_values[:rep_lim]
        # write general data in dic
        overview[run]['n_samples'] = len(fits)
        overview[run]['n_timepoints'] = len(timepoints)
        overview[run+' median']['n_samples'] = len(fits)
        overview[run+' median']['n_timepoints'] = len(timepoints)


        # KINETIC DATA
        timepoints = np.array(timepoints)
        fits       = np.array(fits)                
        sv_values  = np.array(sv_values)
        parameter_names = ['kcincaf','kcafpar','kcafbro','kcafphy','kcafdeg','kpardeg','kbrodeg','kphydeg','par0','bro0','phy0','f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24']
        label_names     = []
        for i in range(len(parameters)):
            sigma  = np.sqrt(np.sum((fits[:,i]-parameters[i])**2)/fits.shape[0])
            median = np.median((fits[:,i]-parameters[i])/parameters[i])
            rel_sigma = sigma/parameters[i]*100
            overview[run][parameter_names[i]] = round(rel_sigma,rounder)
            overview[run+' median'][parameter_names[i]] = round(median*100,rounder)

        # SWEAT VOLUME DATA
        sv_rel_diffs = []
        sv_abs_diffs = []
        outlier_number = 0
        for i in range(fits.shape[0]):
            tmp_diffs     = fits[i,11:]-sv_values[i]
            tmp_rel_diffs = tmp_diffs/sv_values[i]
            for i in tmp_rel_diffs:
                if i < 10:
                    sv_rel_diffs.append(i)
                else:
                    outlier_number += 1
            sv_abs_diffs.append(tmp_diffs)
        print(f'Removed {outlier_number} outlier(s) of SV statistics. This is {round(outlier_number/len(np.array(sv_values).flatten())*100,5)} % of the whole dataset.')
        sv_rel_diffs = np.array(sv_rel_diffs).flatten()
        sv_abs_diffs = np.array(sv_abs_diffs).flatten()
        rel_sigma = np.sqrt(np.sum(sv_rel_diffs**2)/len(sv_rel_diffs))
        abs_sigma = np.sqrt(np.sum(sv_abs_diffs**2)/len(sv_abs_diffs))
        median_rel = np.median(sv_rel_diffs)
        overview[run]['sv'] = round(rel_sigma*100,rounder)
        overview[run+' median']['sv'] = round(median_rel*100,rounder)

        # time-dependent SWEAT VOLUME DATA
        outlier_number = 0
        for t in range(timepoints.shape[0]):
            sv_rel_diffs = []
            sv_abs_diffs = []

            for i in range(fits.shape[0]):
                tmp_diffs     = fits[i,11+t]-sv_values[i,t]
                tmp_rel_diffs = tmp_diffs/sv_values[i,t]
                if tmp_rel_diffs < 10:
                    sv_rel_diffs.append(tmp_rel_diffs)
                else:
                    outlier_number += 1
                sv_abs_diffs.append(tmp_diffs)

            sv_rel_diffs = np.array(sv_rel_diffs).flatten()
            sv_abs_diffs = np.array(sv_abs_diffs).flatten()
            rel_sigma = np.sqrt(np.sum(sv_rel_diffs**2)/len(sv_rel_diffs))
            abs_sigma = np.sqrt(np.sum(sv_abs_diffs**2)/len(sv_abs_diffs))
            median_rel = np.median(sv_rel_diffs)
            overview[run]['sv t='+str(timepoints[t])] = round(rel_sigma*100,rounder)
            overview[run+' median']['sv t='+str(timepoints[t])] = round(median_rel*100,rounder)
        #print(f'Removed {outlier_number} outlier(s) of SV statistics. This is {round(outlier_number/len(np.array(sv_abs_diffs).flatten())*100,5)} % of the whole dataset.')
        print(f'Removed {outlier_number} outlier(s) of SV statistics. This is {round(outlier_number/len(np.array(sv_values).flatten())*100,5)} % of the whole dataset.')

        # CONCENTRATION DATA
        for func, name in zip([caf,par,bro,phy],['caf','par','bro','phy']):
            true_conc     = func(timepoints,*parameters)
            fitted_matrix = []
            for i in range(fits.shape[0]):
                fitted = func(timepoints,*fits[i,:11])
                fitted_matrix.append(fitted)
            fitted_matrix = np.array(fitted_matrix)

            rel_sigma_list = []
            median_list    = []
            for i in range(fitted_matrix.shape[1]):
                abs_sigma = np.sqrt(np.sum((fitted_matrix[:,i]-true_conc[i])**2)/fitted_matrix.shape[0])
                rel_err = (fitted_matrix[:,i]-true_conc[i])/true_conc[i]
                median_list.append(rel_err)
                rel_sigma = abs_sigma/true_conc[i]*100
                rel_sigma_list.append(rel_sigma)
                overview[run][name+' conc. t='+str(timepoints[i])] = round(np.nanmean(rel_sigma),rounder)
                overview[run+' median'][name+' conc. t='+str(timepoints[i])] = round(np.nanmedian(rel_err)*100,rounder)
            if overview[run]['c0'] == '          yes':
                overview[run][name+' conc.'] = round(np.nanmean(rel_sigma_list),rounder)
            else:
                overview[run][name+' conc.'] = round(np.nanmean(rel_sigma_list[1:]),rounder)
            median = np.nanmedian(median_list)
            overview[run+' median'][name+' conc.'] = round(median*100,rounder)

    overview_df = pd.DataFrame(overview)
    #overview_df.to_csv('analysis_tables/later_measurements.csv')
    return overview_df

In [7]:
# earlier measurments with 15\dagger timepoints
later_measurements = ['run_134']
# later maeasurements with 20 timepoints
earlier_measurements   = ['run_139']

extract_quality_parameters_time(later_measurements)

Removed 7 outlier(s) of SV statistics. This is 0.01167 % of the whole dataset.
Removed 7 outlier(s) of SV statistics. This is 0.01167 % of the whole dataset.


/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in true_divide
/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: invalid value encountered in double_scalars
/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/ipykernel_launcher.py:131: RuntimeWarning: Mean of empty slice
/home/users/mgotsmy/envs/robust_loss/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


,run_134,run_134 median
mc_replicates,100,100
c0,yes,yes
loss,max_robust,max_robust
error%,10,10
n_samples,3000,3000
...,...,...
phy conc. t=12.0,16.75,3.15
phy conc. t=13.0,18.1,3.05
phy conc. t=14.0,19.49,3.04
phy conc. t=24.0,35.68,4.2
